#Global Climate Trends

##Analysis


In [1]:
# Dependencies and Setup
import pandas as pd
import os
import csv
import matplotlib.pyplot as plt
import scipy.stats as st

In [2]:
# Study co2 data file
co2_path = "Data/Sorted/co2_emissions.csv"

# Path to input files
co2_input_file_path = os.path.join(co2_path)

# Path to output files
co2_output_file_path = os.path.join( "Data/output/Energy_CO2_Emissions_by_year_n_country.csv")

# Read the energyata and the study results
co2_df = pd.read_csv(co2_input_file_path)
co2_df


,Country,1985,1986,1987,1988,1989,1990,1991,1992,1993,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Canada,403.5,393.4,406.7,437.4,457.4,444.8,428.0,441.9,438.4,...,559.3,564.5,564.3,545.9,561.5,571.5,558.8,500.1,510.2,519.7
1,Mexico,233.9,230.4,239.9,241.0,253.9,280.8,287.3,290.9,293.6,...,496.7,482.8,480.9,491.1,503.3,489.0,480.1,417.8,449.8,505.9
2,US,4576.0,4598.2,4749.9,4969.1,5058.1,4970.5,4922.6,5004.9,5117.6,...,5246.6,5251.7,5137.5,5038.0,4978.8,5133.2,4981.6,4462.7,4768.4,4825.8
3,Argentina,94.6,100.0,104.6,109.9,107.2,100.3,102.8,108.9,107.7,...,187.6,187.8,191.2,190.1,188.5,186.0,174.6,162.3,184.7,190.1
4,Brazil,185.0,199.4,205.8,207.6,211.4,206.4,213.4,212.4,222.5,...,487.0,511.7,494.8,455.9,461.7,442.1,435.5,408.4,461.6,443.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,Sri Lanka,4.9,5.0,5.6,5.2,4.8,5.4,5.4,6.1,6.1,...,15.9,19.6,21.6,24.9,24.5,23.5,25.2,24.7,23.0,19.9
87,Taiwan,73.7,85.5,90.6,106.8,116.8,120.1,128.4,138.3,149.7,...,268.5,275.4,275.7,280.9,286.9,284.8,279.3,265.7,280.2,272.4
88,Thailand,46.1,47.8,56.4,63.8,74.0,89.4,99.1,108.7,122.5,...,265.8,273.7,281.1,286.8,287.5,293.2,288.4,273.1,270.3,275.3
89,Vietnam,17.4,19.0,21.5,21.0,17.9,18.0,17.5,18.4,20.7,...,134.0,150.2,181.9,197.3,199.0,240.4,291.1,285.0,273.1,270.0


In [3]:
# Path to capita file
per_capita_file_path = "Data/Sorted/energy_consumption_per_capita.csv"

# Path to input files
per_capita_input_file_path = os.path.join(per_capita_file_path)

# Path to output files
per_capita_output_file_path = os.path.join("Data/output/primary_energy_consumption_capita_by_year_n_country.csv")

# Read the energyata and the study results
capita_df = pd.read_csv(per_capita_input_file_path)
capita_df

,Country,1985,1986,1987,1988,1989,1990,1991,1992,1993,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Canada,398.0,397.7,405.5,415.8,413.2,391.6,388.2,393.9,399.3,...,410.6,407.5,406.4,396.7,397.8,396.2,388.7,361.8,362.7,367.8
1,Mexico,54.5,52.0,52.8,52.3,54.3,58.1,58.6,58.6,57.6,...,68.8,67.5,66.0,66.7,67.3,65.8,64.5,59.0,63.1,68.4
2,US,309.0,308.8,316.0,327.1,331.4,328.0,323.1,323.2,325.4,...,290.0,290.6,285.5,283.0,281.8,290.1,286.2,263.7,277.2,283.5
3,Argentina,57.2,59.6,61.3,61.4,58.4,56.6,56.7,60.0,61.1,...,83.3,82.4,83.0,81.8,81.0,79.8,74.6,69.5,76.8,79.2
4,Brazil,36.1,37.3,37.6,38.0,38.3,37.4,38.0,37.9,39.0,...,62.0,63.1,61.7,59.7,59.8,59.5,60.1,57.3,60.0,62.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,Sri Lanka,5.7,6.0,6.1,5.9,5.7,6.3,6.2,6.5,7.0,...,13.5,14.4,16.1,17.6,17.0,17.6,17.9,17.5,17.6,15.7
87,Taiwan,73.3,78.8,84.3,92.1,96.8,102.2,107.1,112.8,118.4,...,203.0,206.7,205.7,206.4,205.5,207.7,203.6,197.5,208.8,200.2
88,Thailand,13.5,14.1,15.9,17.5,20.0,23.1,25.1,27.0,30.2,...,67.7,69.5,70.8,71.9,73.0,75.0,74.9,69.5,69.9,70.6
89,Vietnam,3.7,3.9,4.3,4.2,3.9,4.1,4.1,4.4,5.0,...,26.3,28.7,32.4,34.8,37.1,41.2,45.3,44.9,44.6,46.7


In [4]:
# Create Array to further sort data for merging
co2_df_array = []

# Loop over the columns of the dataframe skipping the first column
for column in co2_df.columns[1:]:
    # Create a new dataframe with the country and the year
    cy_list = co2_df[['Country', column]].copy()

    # Adding a new column to the dataframe and positionong
    cy_list.insert(0, 'Year', column)

    #Rename for dataset
    cy_list.rename(columns={column: "Energy CO2 Emissions" }, inplace=True)

    # Adding the new dataframe to the array
    co2_df_array.append(cy_list)

co2_df_array

[    Year             Country Energy CO2 Emissions
 0   1985              Canada                403.5
 1   1985              Mexico                233.9
 2   1985                  US               4576.0
 3   1985           Argentina                 94.6
 4   1985              Brazil                185.0
 ..   ...                 ...                  ...
 86  1985           Sri Lanka                  4.9
 87  1985              Taiwan                 73.7
 88  1985            Thailand                 46.1
 89  1985             Vietnam                 17.4
 90  1985  Other Asia Pacific                166.7
 
 [91 rows x 3 columns],
     Year             Country Energy CO2 Emissions
 0   1986              Canada                393.4
 1   1986              Mexico                230.4
 2   1986                  US               4598.2
 3   1986           Argentina                100.0
 4   1986              Brazil                199.4
 ..   ...                 ...                  ...
 86  

In [5]:
# Merge dfs 
merged_co2_df = pd.concat(co2_df_array, ignore_index=True)
merged_co2_df

#Save csv
# merged_co2_df.to_csv(co2_output_file_path, index=False, header=True)

,Year,Country,Energy CO2 Emissions
0,1985,Canada,403.5
1,1985,Mexico,233.9
2,1985,US,4576.0
3,1985,Argentina,94.6
4,1985,Brazil,185.0
...,...,...,...
3453,2022,Sri Lanka,19.9
3454,2022,Taiwan,272.4
3455,2022,Thailand,275.3
3456,2022,Vietnam,270.0


In [6]:
capita_df_array = []

# Loop over the columns of the dataframe skipping the first column
for column in capita_df.columns[1:]:
    # Create a new dataframe with the country and the year
    cy_list = capita_df[['Country', column]].copy()

    # Adding a new column to the dataframe and positionong
    cy_list.insert(0, 'Year', column)

    #Rename for dataset
    cy_list.rename(columns={column: 'Primary Energy Consumption per Capita'}, inplace=True)

    # Adding the new dataframe to the array
    capita_df_array.append(cy_list)

capita_df_array

[    Year             Country  Primary Energy Consumption per Capita
 0   1985              Canada                                  398.0
 1   1985              Mexico                                   54.5
 2   1985                  US                                  309.0
 3   1985           Argentina                                   57.2
 4   1985              Brazil                                   36.1
 ..   ...                 ...                                    ...
 86  1985           Sri Lanka                                    5.7
 87  1985              Taiwan                                   73.3
 88  1985            Thailand                                   13.5
 89  1985             Vietnam                                    3.7
 90  1985  Other Asia Pacific                                   19.9
 
 [91 rows x 3 columns],
     Year             Country  Primary Energy Consumption per Capita
 0   1986              Canada                                  397.7
 1   198

In [7]:
# Merge dfs 
merged_capita_df = pd.concat(capita_df_array, ignore_index=True)
merged_capita_df

#Save csv
merged_capita_df.to_csv(per_capita_output_file_path, index=False, header=True)

In [8]:
# Path to folders for merging of all 3 cleaned datasets
input_folder_path = os.path.join(".", "Data", "Input")
output_folder_path = os.path.join(".", "Data", "output")

# Path to input files
total_consumption_input_file_path = os.path.join(input_folder_path, "primary_energy_consumption.csv")
per_capita_input_file_path = os.path.join(input_folder_path, "energy_consumption_capita.csv")
co2_input_file_path = os.path.join(input_folder_path, "co2_emissions_country.csv")

In [9]:
# Read input files
df_consumption = pd.read_csv(total_consumption_input_file_path)
df_per_capita = pd.read_csv(per_capita_input_file_path)
df_co2 = pd.read_csv(co2_input_file_path)

In [10]:
print(df_consumption.columns)

Index(['Year', 'Country', 'Primary Energy Consumption'], dtype='object')


In [11]:
merged_df = pd.merge(df_consumption, df_per_capita, on= ['Year','Country'], how='inner')
# merged_df

energy_consumption_df = pd.merge(merged_df, df_co2, on= ['Year','Country'], how='inner')
energy_consumption_df

,Year,Country,Primary Energy Consumption,Primary Energy Consumption per Capita,Energy CO2 Emissions
0,1985,Canada,10.29,398.0,403.5
1,1985,Mexico,4.08,54.5,233.9
2,1985,US,72.66,309.0,4576.0
3,1985,Argentina,1.73,57.2,94.6
4,1985,Brazil,4.94,36.1,185.0
...,...,...,...,...,...
3453,2022,Sri Lanka,0.34,15.7,19.9
3454,2022,Taiwan,4.78,200.2,272.4
3455,2022,Thailand,5.06,70.6,275.3
3456,2022,Vietnam,4.59,46.7,270.0
